In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
import numpy as np
import pandas as pd

from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal

from sklearn.datasets import make_classification

import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
np.random.seed(0)

In [3]:
train = pd.read_csv('../input/train.csv', index_col='id')
test = pd.read_csv('../input/test.csv', index_col='id')
data = pd.concat([train, test], sort=False)

In [4]:
methods = ['diff', 'discrete']
pred = pd.DataFrame(index=test.index, columns=methods)

for i in tqdm_notebook(range(512)):
    
    cur = data[data['wheezy-copper-turtle-magic']==i]
    train_idx = [i for i in cur.index if i in train.index]
    test_idx = [i for i in cur.index if i in test.index]
    
    y_data = cur['target']
    x_data = cur.loc[:, cur.std()>2]
    cur_test = x_data.loc[test_idx]
    
    gm = GaussianMixture(6, n_init=20, max_iter=10000, tol=1e-5).fit(x_data)
    tmp = pd.DataFrame([gm.predict(x_data), y_data.values]).dropna(axis=1)
    pos_val_counts = tmp.loc[:, tmp.loc[1]==1].loc[0].value_counts()
    neg_val_counts = tmp.loc[:, tmp.loc[1]==0].loc[0].value_counts()
    pos_cluster = pos_val_counts.iloc[:3].index.astype(int)
    neg_cluster = neg_val_counts.iloc[:3].index.astype(int)
    pred.loc[cur_test.index, 'discrete'] = pd.Series(gm.predict(cur_test)).replace(pos_cluster, -1).replace(neg_cluster, 0).abs().values



In [5]:
pred = pred.reset_index()

In [6]:
sub = pd.read_csv('../input/sample_submission.csv')
sub['target'] = pred['discrete']
sub.to_csv('submission.csv',index=False)